# Generating adversarial examples using FGSM (PFN coding interview 2018)

This is my solution to the 2018 coding interview at Preferred Networks. (https://github.com/pfnet/intern-coding-tasks/tree/master/2018/ml)

I did not apply for the internship, but found the problem interesting and wanted to solve it. The goal is to use the Fast Gradient Sign Method to generate adversarial examples of 32x32 pixel images of hand-written greek symbols.

A key part of the challenge is that the use of external modules are forbidden, so we have to define all our functions ourselves. Since this is a notebook, and I want to explain each function while we define them, I will not create any classes to make the readability easier. For any real application, creating more compact classes is advisable.

### Task description (from PFN interview)
This task is about adversarial examples. You are given an image dataset of
hand-written characters and a predictor (that is, a classifier) for the dataset.
The objective is to modify the original input image by adding a small amount
of noise so that the predictor will missclassify it. That is, we want to make an
input image that will deceive the predictor.

Back-story: Thanks to the advancement of machine learning and deep learning,
prediction models are going to be used in important applications, such as
autonomous driving and image authentication. One motivation for adversarial
examples is considering the risk that malicious users may abuse such predictors.


# Problem 1: Helper functions without external modules
Problem 1 is just to design helper functions for basic linear algebra methods, since we will use them in the later problems.

In [87]:
class UnitTests():
    
    def vec_dim_check(self,x,y):
        if len(x) != len(y):
            print("ERROR: Dimension of vectors does not match.")
            return False
        else:
            return True
        
    def vec_elem_check(self,x):
        if not all(isinstance(n,(int,float)) for n in x):
            print("ERROR: Vectors can only contain ints or floats.")
            return False
        else:
            return True
        
    def vec_mat_prod_check(self,A,x):
        
        A_dims = (len(A),len(A[0]))
        x_dims = len(x)
    
        if x_dims != A_dims[1]:
            print("ERROR: Dimension of matrix and vector does not match")
            return False
        else:
            return True
        

In [88]:
UT = UnitTests()

In [89]:
""" This function calculates the sum of two vectors of equal dimensions. """

def vec_sum(x,y):
    
    if not UT.vec_elem_check(x):
        return
    if not UT.vec_dim_check(x,y):
        return
    
    return list(map(lambda x,y: x+y, x,y))

In [6]:
x1, y1 = [1,2,3], [1,2,3]
x2, y2 = ["a",2,3], [1,2,3]
x3, y3 = [1,2,3,4], [1,2,3]

print(vec_sum(x1,y1))
print(vec_sum(x2,y2))
print(vec_sum(x3,y3))

[2, 4, 6]
ERROR: Vectors can only contain ints or floats.
None
ERROR: Dimension of vectors does not match.
None


In [55]:
""" This function calculates the matrix-vector product."""

def vec_mat_prod(A, x):
    
    A_dims = (len(A),len(A[0]))
    x_dims = len(x)
    
    if x_dims != A_dims[1]:
        print("ERROR: Dimension of matrix and vector does not match")
    
    y = [[] for n in range(A_dims[0])]
    
    for i in range(A_dims[0]):
        y[i] = sum(A[i][j]*x[j] for j in range(x_dims))
        
    return y
    

In [91]:
A = [[1,2,3],[4,5,6]]
x = [1,2,3]
vec_mat_prod(A,x)

[14, 32]

In [102]:
""" This function calculates the matrix transpose of input matrix A. Has to be a mutli-column matrix."""
def mat_transpose(A):
        
    row_dim = len(A)
    col_dim = len(A[0])
    
    A_T = [[A[i][j] for i in range(row_dim)] for j in range(col_dim)]
    return A_T
            

In [104]:
print(A)
print(mat_transpose(A))

[[1, 2, 3], [4, 5, 6]]
[[1, 4], [2, 5], [3, 6]]


In [105]:
""" This function applies elementwise rectified linear unit on a vector. """
def ReLU(x):

    if not UT.vec_elem_check(x):
        return
    
    x = [max(0,x[i]) for i in range(len(x))]

    return x

In [106]:
ReLU([-1,2,3])

[0, 2, 3]

In [107]:
""" This function applies a normalized exponential transformation on a vector (i.e. softmax). """
def softMax(x):
    
    e = 2.7182818284
    
    exp_vector = [e**(x[n]) for n in range(len(x))]
    exp_sum = sum(exp_vector)
    
    exp_vector = [exp_vector[n]/exp_sum for n in range(len(x))]
    
    return exp_vector

In [108]:
x = [-1,0,1]
print(softMax(x))
print(f"Sum is normalized to {sum(softMax(x))}")

[0.09003057317346094, 0.24472847105785542, 0.6652409557686837]
Sum is normalized to 1.0


# Problem 2a: Importing and converting data

In [261]:
""" This is a simple function to load the pgm image files, which are 32x32 pixel. We unwrap them and store them
    in a 32*32 = 1024 dim vector. Full path of files is given, because the use of external modules like os and sys 
    is not allowed. The first three lines of the pgm files is always
    
    P2
    32 32
    255
    
    and then the following lines are the pixel values in grayscale from 0 to 255. """

def read_pgm(n,key = "normal"):
    path_dict = {"normal":"/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/pgm/",
                 "adverse":"/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/pgm_ad/",
                 "baseline":"/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/pgm_bl_ad/"}
    
    path = path_dict[key]
    file = open(path + str(n) + ".pgm","r")
    data = file.read().split()
    file.close()
    
    fig = data[4:]
    fig_norm = [float(fig[n])/255 for n in range(len(fig))]
    
    return fig_norm

In [110]:
def convert_to_matrix(A):
    n_rows = len(A)
    mat = []
    
    for n in range(n_rows):
        mat.append(A[n].split())
    
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            mat[i][j] = float(mat[i][j])
        
    return mat

In [111]:
def convert_to_vector(x):
    for n in range(len(x)):
        x[n] = float(x[n])
    return x

In [301]:
""" In this task, we are given the parameters for a pre-trained network. The network arcitechture
    is hidden layers of 256 neurons, and then 23 neurons in the output layer. Each of the 23 output neurons
    corresponds to the 23 labels we have for the different types of greek symbols.
    The parameters are stored in a txt file, where the first 256 lines is the weights of the first hidden layer,
    then the next line is the biases of the first hidden layer, then 256 lines of the second hidden layer etc."""

def load_network_parameters(file_loc):

    file = open(file_loc,"r")
    data = file.readlines()
    file.close()
    
    H = 256
    C = 23

    W1 = data[:H]
    W1 = convert_to_matrix(W1)

    b1 = data[H].split()
    b1 = convert_to_vector(b1)

    W2 = data[H+1:2*H+1]
    W2 = convert_to_matrix(W2)

    b2 = data[2*H+1].split()
    b1 = convert_to_vector(b2)

    W3 = data[2*H+2:2*H + 2 + C]
    W3 = convert_to_matrix(W3)

    b3 = data[-1].split()
    b3 = convert_to_vector(b3)
    
    return W1,b1,W2,b2,W3,b3


In [302]:
file_loc = "/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/param.txt"
W1,b1,W2,b2,W3,b3 = load_network_parameters(file_loc)

In [303]:
""" Here we import the labels and store them in a vector. """

file = open("/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/labels.txt","r")
labels = file.read().split()
file.close()
labels = convert_to_vector(labels)

# Problem 2b: Using the predictor model to classify greek symbols

In [304]:
""" The predict function takes the image vectors as input, and runs them through the neural network with 
    pretrained weights. The output layer is a softmax layer where the index of the largest element is the 
    networks prediction of the label (which greek character the image represents)."""

def predict(x):
    a1 = vec_sum(vec_mat_prod(W1,x),b1)
    h1 = ReLU(a1)
    a2 = vec_sum(vec_mat_prod(W2,h1),b2)
    h2 = ReLU(a2)
    y  = vec_sum(vec_mat_prod(W3,h2),b3)
    f = softMax(y)
    return f, float(f.index(max(f))+1)

In [305]:
""" Here we run through all the images, and compare the networks prediction with the true labels."""

correct = 0

for n in range(1,155):
    x = read_pmg(n)
    _, pred = predict(x)
    true = labels[n-1]
    
    if true == pred:
        correct += 1


In [306]:
accuracy = correct/len(labels)
print(f"The accuracy of the model is {100*accuracy:.1f}%")

The accuracy of the model is 85.1%


# Problem 3a: Implementing Fast Gradient Sign Method

In [201]:
def Backward(x,y):
    
    if not UT.vec_dim_check(x,y):
        return
    if not UT.vec_elem_check(x) and UT.vec_elem_check(y):
        return

    vec = []
    for i in range(len(x)):
        if y[i] > 0:
            vec.append(x[i])
        else:
            vec.append(0)
    return vec

In [202]:
a = Backward([1,2,3,4],[-1,0,1/1e10,1e10])
a

[0, 0, 3, 4]

In [203]:
def sign(x,e_0):
    x = [+1.0*e_0 if elem > 0 else -1.0*e_0 for elem in x]
    return x

In [275]:
sign([-1,0,1],2)

[-2.0, -2.0, 2.0]

In [321]:
a = [1,2,3]
a.index(max(a))

2

In [78]:
def random_vec(seed,length):
    X = seed
    a = 1664525
    b = 1013904223
    m = 2**32
    vec = []
    
    for n in range(length):
        X = (a*X+b)%m
        vec.append(X/m-0.5)
        
    return vec

In [79]:
random_vec(3,3)

[-0.26276936987414956, 0.050678204046562314, 0.3736585769802332]

In [324]:
def predict_adverse(x,e_0, label, baseline = False, seed = 1234):
    
    a1 = vec_sum(vec_mat_prod(W1,x),b1)
    h1 = ReLU(a1)
    a2 = vec_sum(vec_mat_prod(W2,h1),b2)
    h2 = ReLU(a2)
    y  = vec_sum(vec_mat_prod(W3,h2),b3)
    f = softMax(y)
    
    dt = [0.0]*len(f)
    dt[int(label)-1] = -1.0    
    
    D_y_L = vec_sum(dt,f)
    D_h2_L = vec_mat_prod(mat_transpose(W3),D_y_L)
    D_a2_L = Backward(D_h2_L,a2)
    D_h1_L = vec_mat_prod(mat_transpose(W2), D_a2_L)
    D_a1_L = Backward(D_h1_L,a1)
    D_x_L = vec_mat_prod(mat_transpose(W1), D_a1_L)
    
    if baseline == True:
        rand_vec = random_vec(seed,length=len(x))
        bl_epsilon = sign(rand_vec,e_0)
        x_bl = vec_sum(x,bl_epsilon)
        x_bl = [int((elem + e_0)*(255/(1+2*e_0))) for elem in x_bl]
        return x_bl
    else:
        epsilon = sign(D_x_L,e_0)
        x_ad = vec_sum(x,epsilon)
        x_ad = [int((elem + e_0)*(255/(1+2*e_0))) for elem in x_ad]
        return x_ad

In [259]:
def save_ad_fig(x_ad,n,baseline = False):
    if baseline == False:
        path = "/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/pgm_ad/"
    else:
        path = "/home/vegardbs/Machine Learning/intern-coding-tasks-master/2018/ml/pgm_bl_ad/"

    file = open(path + str(n) + ".pgm","w+")    
    file.write("P2\n")
    file.write("32 32\n")
    file.write("255\n")
    
    k = 0
    for i in range(32):
        for j in range(32):
            file.write(str(x_ad[k]) + " ")
            k += 1
        file.write("\n")
    file.close()

In [326]:
""" Here we create adversal figures """

for n in range(1,len(labels)+1):
    e_0 = 0.1

    x = read_pgm(n)
    label = labels[n-1]
    x_ad = predict_adverse(x,e_0,label,baseline=True)
    save_ad_fig(x_ad,n,baseline=True)

In [327]:
""" Here we try to predict the adversal images, and compare the networks prediction with the true labels."""

correct = 0

for n in range(1,len(labels)+1):
    x = read_pgm(n,key = "adverse")
    _, pred = predict(x)
    true = labels[n-1]
    if true == pred:
        correct += 1


In [328]:
accuracy = correct/len(labels)
print(f"The accuracy of the model is {100*accuracy:.1f}%")

The accuracy of the model is 0.6%


In [329]:
""" Here we run through the the adversal images created by adding random noise (the baseline), 
    and compare the networks prediction with the true labels."""

correct = 0

for n in range(1,len(labels)+1):
    x = read_pgm(n,key = "baseline")
    _, pred = predict(x)
    true = labels[n-1]
    
    if true == pred:
        correct += 1


In [330]:
accuracy = correct/len(labels)
print(f"The accuracy of the model is {100*accuracy:.1f}%")

The accuracy of the model is 81.8%
